## Spark DataFrames

In [4]:
import findspark
findspark.init('/home/spark/spark-2.4.0-bin-hadoop2.7')

### Aula 1

In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [8]:
df = spark.read.json('people.json')

In [9]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [10]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [11]:
df.describe()

DataFrame[summary: string, age: string, name: string]

In [14]:
df.columns

['age', 'name']

In [13]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [17]:
# Caso seja necessário alterar o schema de um dataset
from pyspark.sql.types import StructField, StringType, StructType, IntegerType

data_schema = [StructField('age', IntegerType(), True), StructField('name', StringType(), True)]
final_struct = StructType(fields=data_schema)
df = spark.read.json('people.json', schema=final_struct)
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



### Aula 2

In [18]:
df['age']

Column<b'age'>

In [19]:
type(df['age'])

pyspark.sql.column.Column

In [21]:
df.select('age')

DataFrame[age: int]

In [22]:
type(df.select('age'))

pyspark.sql.dataframe.DataFrame

In [23]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [24]:
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [26]:
type(df.head(2)[0])

pyspark.sql.types.Row

In [27]:
df.select(['age', 'name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [36]:
# Não é inplace
df.withColumn('double_age', df['age']*2).show()

+----+-------+----------+
| age|   name|double_age|
+----+-------+----------+
|null|Michael|      null|
|  30|   Andy|        60|
|  19| Justin|        38|
+----+-------+----------+



In [38]:
# Não é inplace
df.withColumnRenamed('age', 'age_').show()

+----+-------+
|age_|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [39]:
df.createOrReplaceTempView('people')

In [40]:
results = spark.sql('SELECT * FROM people')
results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [41]:
results = spark.sql('SELECT * FROM people where age=30')
results.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



### Aula 3 - Operations

In [42]:
spark = SparkSession.builder.appName('Ops').getOrCreate()

In [43]:
df = spark.read.csv('appl_stock.csv', inferSchema=True, header=True)

In [44]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [45]:
df.show()

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|               Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:00|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|    

In [46]:
df.filter('Close < 500').show()

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|               Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:00|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|    

In [47]:
df.filter('Close < 500').select('Open').show()

+------------------+
|              Open|
+------------------+
|        213.429998|
|        214.599998|
|        214.379993|
|            211.75|
|        210.299994|
|212.79999700000002|
|209.18999499999998|
|        207.870005|
|210.11000299999998|
|210.92999500000002|
|        208.330002|
|        214.910006|
|        212.079994|
|206.78000600000001|
|202.51000200000001|
|205.95000100000001|
|        206.849995|
|        204.930004|
|        201.079996|
|192.36999699999998|
+------------------+
only showing top 20 rows



In [48]:
df.filter(df['Close'] < 500).select(['Open', 'Close']).show()

+------------------+------------------+
|              Open|             Close|
+------------------+------------------+
|        213.429998|        214.009998|
|        214.599998|        214.379993|
|        214.379993|        210.969995|
|            211.75|            210.58|
|        210.299994|211.98000499999998|
|212.79999700000002|210.11000299999998|
|209.18999499999998|        207.720001|
|        207.870005|        210.650002|
|210.11000299999998|            209.43|
|210.92999500000002|            205.93|
|        208.330002|        215.039995|
|        214.910006|            211.73|
|        212.079994|        208.069996|
|206.78000600000001|            197.75|
|202.51000200000001|        203.070002|
|205.95000100000001|        205.940001|
|        206.849995|        207.880005|
|        204.930004|        199.289995|
|        201.079996|        192.060003|
|192.36999699999998|        194.729998|
+------------------+------------------+
only showing top 20 rows



In [49]:
df.filter((df['Close'] < 500) & (df['Open'] > 500)).select(['Open', 'Close']).show()

+----------+------------------+
|      Open|             Close|
+----------+------------------+
|514.259995|        497.669975|
|500.250008|495.26997400000005|
|506.199997|494.63999900000005|
|502.539993|        499.779984|
+----------+------------------+



In [50]:
df.filter((df['Close'] < 500) & ~(df['Open'] > 500)).select(['Open', 'Close']).show()

+------------------+------------------+
|              Open|             Close|
+------------------+------------------+
|        213.429998|        214.009998|
|        214.599998|        214.379993|
|        214.379993|        210.969995|
|            211.75|            210.58|
|        210.299994|211.98000499999998|
|212.79999700000002|210.11000299999998|
|209.18999499999998|        207.720001|
|        207.870005|        210.650002|
|210.11000299999998|            209.43|
|210.92999500000002|            205.93|
|        208.330002|        215.039995|
|        214.910006|            211.73|
|        212.079994|        208.069996|
|206.78000600000001|            197.75|
|202.51000200000001|        203.070002|
|205.95000100000001|        205.940001|
|        206.849995|        207.880005|
|        204.930004|        199.289995|
|        201.079996|        192.060003|
|192.36999699999998|        194.729998|
+------------------+------------------+
only showing top 20 rows



In [52]:
df.filter(df['Low'] == 197.16).select('Volume').show()

+---------+
|   Volume|
+---------+
|220441900|
+---------+



In [54]:
result = df.filter(df['Low'] == 197.16).collect()
result

[Row(Date=datetime.datetime(2010, 1, 22, 0, 0), Open=206.78000600000001, High=207.499996, Low=197.16, Close=197.75, Volume=220441900, Adj Close=25.620401)]

In [55]:
type(result)

list

In [57]:
result[0].asDict()

{'Adj Close': 25.620401,
 'Close': 197.75,
 'Date': datetime.datetime(2010, 1, 22, 0, 0),
 'High': 207.499996,
 'Low': 197.16,
 'Open': 206.78000600000001,
 'Volume': 220441900}

In [58]:
result[0].asDict()['High']

207.499996

### Aula 4 - GroupBy and Aggregate

In [59]:
spark = SparkSession.builder.appName('Aggs').getOrCreate()

In [60]:
df = spark.read.csv('sales_info.csv', inferSchema=True, header=True)

In [61]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [62]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [64]:
df.groupBy('Company')

In [65]:
df.groupBy('Company').mean()

DataFrame[Company: string, avg(Sales): double]

In [66]:
df.groupBy('Company').mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [69]:
df.groupBy('Company').sum().show()
df.groupBy('Company').max().show()
df.groupBy('Company').min().show()
df.groupBy('Company').count().show()

+-------+----------+
|Company|sum(Sales)|
+-------+----------+
|   APPL|    1480.0|
|   GOOG|     660.0|
|     FB|    1220.0|
|   MSFT|     967.0|
+-------+----------+

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+

+-------+----------+
|Company|min(Sales)|
+-------+----------+
|   APPL|     130.0|
|   GOOG|     120.0|
|     FB|     350.0|
|   MSFT|     124.0|
+-------+----------+

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



In [70]:
df.agg({'Sales': 'max'}).show()

+----------+
|max(Sales)|
+----------+
|     870.0|
+----------+



In [72]:
group_data = df.groupBy('Company')
group_data.agg({'Sales': 'mean'}).show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [73]:
from pyspark.sql.functions import countDistinct, avg, stddev

In [78]:
df.select(countDistinct('Sales')).show()
df.select(avg('Sales')).show()
df.select(stddev('Sales').alias('std')).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+

+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+

+------------------+
|               std|
+------------------+
|250.08742410799007|
+------------------+



In [79]:
from pyspark.sql.functions import format_number

In [84]:
sales_std = df.select(stddev('Sales'))
sales_std.select(format_number('stddev_samp(Sales)', 2).alias('std')).show()

+------+
|   std|
+------+
|250.09|
+------+



In [85]:
df.orderBy('Sales').show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [86]:
df.orderBy(df['Sales'].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



### Aula 5 

In [87]:
spark = SparkSession.builder.appName('Miss').getOrCreate()

In [88]:
df = spark.read.csv('ContainsNull.csv', inferSchema=True, header=True)

In [89]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [94]:
df.na.drop().show()
df.na.drop(thresh=2).show()
df.na.drop(how='any').show()
df.na.drop(how='all').show()
df.na.drop(subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [96]:
df.na.fill('No Name').show()
df.na.fill(0).show()
df.na.fill('No Value', subset=['Name']).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| null|
|emp2|No Name| null|
|emp3|No Name|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

+----+--------+-----+
|  Id|    Name|Sales|
+----+--------+-----+
|emp1|    John| null|
|emp2|No Value| null|
|emp3|No Value|345.0|
|emp4|   Cindy|456.0|
+----+--------+-----+



In [97]:
from pyspark.sql.functions import mean

In [103]:
mean_val = df.select(mean('Sales'))
mean_val.show()
mean_val = df.select(mean(df['Sales'])).collect()
mean_val

+----------+
|avg(Sales)|
+----------+
|     400.5|
+----------+



[Row(avg(Sales)=400.5)]

In [104]:
df.na.fill(df.select(mean(df['Sales'])).collect()[0][0], subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### Aula 6 - Dates and Timestamps

In [105]:
spark = SparkSession.builder.appName('Dates').getOrCreate()

In [106]:
df = spark.read.csv('appl_stock.csv', inferSchema=True, header=True)

In [110]:
from pyspark.sql.functions import (dayofmonth, dayofweek, dayofyear, hour, minute, second, month, year, 
                                   weekofyear, format_number, date_format)

In [111]:
df.select(dayofmonth(df['Date'])).show()
df.select(dayofweek(df['Date'])).show()
df.select(dayofyear(df['Date'])).show()
df.select(month(df['Date'])).show()
df.select(year(df['Date'])).show()
df.select(hour(df['Date'])).show()
df.select(minute(df['Date'])).show()
df.select(second(df['Date'])).show()

+----------------+
|dayofmonth(Date)|
+----------------+
|               4|
|               5|
|               6|
|               7|
|               8|
|              11|
|              12|
|              13|
|              14|
|              15|
|              19|
|              20|
|              21|
|              22|
|              25|
|              26|
|              27|
|              28|
|              29|
|               1|
+----------------+
only showing top 20 rows

+---------------+
|dayofweek(Date)|
+---------------+
|              2|
|              3|
|              4|
|              5|
|              6|
|              2|
|              3|
|              4|
|              5|
|              6|
|              3|
|              4|
|              5|
|              6|
|              2|
|              3|
|              4|
|              5|
|              6|
|              2|
+---------------+
only showing top 20 rows

+---------------+
|dayofyear(Date)|
+---------------+
|     

In [114]:
new_df = df.withColumn('Year', year(df['Date']))

In [116]:
new_df.groupBy('Year').mean().show()

+----+------------------+------------------+------------------+------------------+--------------------+------------------+---------+
|Year|         avg(Open)|         avg(High)|          avg(Low)|        avg(Close)|         avg(Volume)|    avg(Adj Close)|avg(Year)|
+----+------------------+------------------+------------------+------------------+--------------------+------------------+---------+
|2015|120.17575393253965|121.24452385714291| 118.8630954325397|120.03999980555547|  5.18378869047619E7|115.96740080555561|   2015.0|
|2013| 473.1281355634922| 477.6389272301587|468.24710264682557| 472.6348802857143|          1.016087E8| 62.61798788492063|   2013.0|
|2014| 295.1426195357143|297.56103184523823| 292.9949599801587| 295.4023416507935| 6.315273055555555E7| 87.63583323809523|   2014.0|
|2012|     576.652720788| 581.8254008040001| 569.9211606079999| 576.0497195640002|       1.319642044E8| 74.81383696800002|   2012.0|
|2016|104.50777772619044| 105.4271825436508|103.69027771825397|104.60

In [117]:
new_df.groupBy('Year').mean().select(['Year', 'avg(Open)']).show()

+----+------------------+
|Year|         avg(Open)|
+----+------------------+
|2015|120.17575393253965|
|2013| 473.1281355634922|
|2014| 295.1426195357143|
|2012|     576.652720788|
|2016|104.50777772619044|
|2010| 259.9576190992064|
|2011|364.06142773412705|
+----+------------------+



In [127]:
new_df.groupBy('Year').mean().select(['Year', format_number('avg(Open)', 2).alias('avg(Open)')]).show()

+----+---------+
|Year|avg(Open)|
+----+---------+
|2015|   120.18|
|2013|   473.13|
|2014|   295.14|
|2012|   576.65|
|2016|   104.51|
|2010|   259.96|
|2011|   364.06|
+----+---------+

